In [2]:
import numpy as np
import pandas as pd

import re, os, string, time, unicodedata

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk import word_tokenize, sent_tokenize

import inflect
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.metrics import (
    ConfusionMatrixDisplay, RocCurveDisplay,
    roc_auc_score, precision_score, recall_score, f1_score
)
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline, make_pipeline

import contractions
import seaborn as sns; sns.set()

import matplotlib.pyplot as plt
import timeit

In [11]:
df = pd.read_csv("Data/Datos_proyecto_II_BI_2021.csv", encoding="'latin-1'")

C:\Users\juank\AppData\Local\Temp\ipykernel_22204\4158079951.py:1: DtypeWarning: Columns (334,410,412,540,541,543,545,549,552,553,554,555,556,557) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Data/Datos_proyecto_II_BI_2021.csv", encoding="'latin-1'")


In [12]:
df

,Unnamed: 0,DIRECTORIO,DIRECTORIO_HOG,DIRECTORIO_PER,DPTO,MPIO,CLASE,COD_LOCALIDAD,NOMBRE_LOCALIDAD,COD_UPZ_GRUPO,...,NPCHP35C,NPCHP35D,NPCHP35E,NPCHP35F,NPCHP35I,NPCHP35J,NPCHP35K,NPCHP36,NPCHP36A,NPCHP37
0,1,1000001,10000011,100000111,11,11001,1,7.0,Bosa,87.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN
1,2,1000004,10000041,100000412,11,11001,1,7.0,Bosa,87.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN
2,3,1000033,10000331,100003313,11,11001,1,5.0,Usme,58.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
3,4,1000040,10000401,100004011,11,11001,1,9.0,Fontibon,77.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN
4,5,1000041,10000411,100004111,11,11001,1,9.0,Fontibon,77.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8625,8626,999973,9999731,99997311,11,11001,1,7.0,Bosa,87.0,...,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN
8626,8627,999974,9999741,99997413,11,11001,1,7.0,Bosa,87.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
8627,8628,999977,9999771,99997713,11,11001,1,7.0,Bosa,87.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8628,8629,999983,9999831,99998313,11,11001,1,7.0,Bosa,85.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN


In [14]:
df.describe()

,Unnamed: 0,DIRECTORIO,DIRECTORIO_HOG,DIRECTORIO_PER,DPTO,MPIO,CLASE,COD_LOCALIDAD,COD_UPZ_GRUPO,ESTRATO2021,...,NPCHP35C,NPCHP35D,NPCHP35E,NPCHP35F,NPCHP35I,NPCHP35J,NPCHP35K,NPCHP36,NPCHP36A,NPCHP37
count,8630.000000,8.630000e+03,8.630000e+03,8.630000e+03,8630.000000,8630.000000,8630.000000,7238.000000,7057.000000,8.630000e+03,...,6971.000000,6971.000000,6971.000000,6971.000000,6971.000000,6971.000000,6971.000000,8430.000000,6.440000e+02,841.000000
mean,4315.500000,1.062837e+06,1.062837e+07,1.066168e+08,13.258169,13329.605330,1.100000,9.801603,165.352274,1.332975e+07,...,1.995410,1.985942,1.969875,1.995553,1.961842,1.991536,1.978626,1.923606,1.919626e+05,2.727705
std,2491.410745,5.848858e+05,5.848858e+06,6.117052e+07,5.149574,5311.175505,0.424466,5.238107,263.488868,5.311165e+06,...,0.067602,0.117740,0.170943,0.066542,0.191592,0.091614,0.144639,0.265643,4.613652e+05,1.321365
min,1.000000,2.201020e+05,2.201021e+06,2.201021e+07,11.000000,11001.000000,1.000000,1.000000,9.000000,1.100110e+07,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000e+00,1.000000
25%,2158.250000,9.878092e+05,9.878094e+06,9.878094e+07,11.000000,11001.000000,1.000000,6.000000,37.000000,1.100113e+07,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0.000000e+00,2.000000
50%,4315.500000,1.042325e+06,1.042325e+07,1.042364e+08,11.000000,11001.000000,1.000000,10.000000,67.000000,1.100116e+07,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0.000000e+00,3.000000
75%,6472.750000,1.145552e+06,1.145553e+07,1.145560e+08,11.000000,11001.000000,1.000000,13.000000,96.000000,1.100119e+07,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,6.250000e+04,3.000000
max,8630.000000,3.006798e+06,3.006798e+07,1.165204e+09,25.000000,25899.000000,3.000000,20.000000,817.000000,2.589910e+07,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000e+06,9.000000


In [15]:
(df.isnull().sum() / df.shape[0]).sort_values(ascending = False)

NPCFP25F      1.000000
NPCHP23A      1.000000
NPCHP25C      1.000000
NPCEP29A      1.000000
NPCHP25F      0.999884
                ...   
NPCFP10D      0.000000
NPCFP10C      0.000000
NPCFP10B      0.000000
NPCFP10A      0.000000
Unnamed: 0    0.000000
Length: 601, dtype: float64